# Import

In [33]:
import os
import re
import random
import numpy as np
import pandas as pd
import string
from tqdm.autonotebook import tqdm, trange

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.optim import lr_scheduler

from sklearn import model_selection
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

import transformers
import tokenizers
from transformers import AdamW, get_linear_schedule_with_warmup

from IPython.core.debugger import set_trace

# Settings

In [34]:
args = {
    'data_dir': '/data/ai_challenge/tweet_sentiment_extraction/',
    'train_file': 'train.csv',
    'test_file': 'test.csv',
    'save_model_dir': 'trained_model',
    
    'model_name': 'bert-base-uncased',
    'do_eval': False,
    'gradient_accumulation_steps': 1,
    'max_seq_length': 128,
    'batch_size': 8,
    'n_epoch': 3,
    'weight_decay': 1e-7,
    'adam_epsilon': 1e-8,
    'warmup_steps': 0,
    'n_splits': 5,
    'learning_rate': 1e-5,
    'seed': 1234,
}

# class config:
#     DATA_DIR = '../input/tweet-sentiment-extraction'
#     BERT_PATH = '../input/bert-base-uncased'
#     OUTPUT_DIR = '/kaggle/working'
#     TRAIN_FILE = 'train.csv'
#     TEST_FILE = 'test.csv'
#     SAVE_MODEL_DIR = ''
#     MAX_LEN = 128
#     TRAIN_BATCH_SIZE = 32
#     VALID_BATCH_SIZE = 32
#     EPOCHS = 3
#     N_SPLITS = 5
#     SEED = 1234
#     TOKENIZER = tokenizers.BertWordPieceTokenizer(
#         f"{BERT_PATH}/vocab.txt", 
#         lowercase=True
#     )

class config:
    DATA_DIR = '/data/ai_challenge/tweet_sentiment_extraction'
    BERT_PATH = '/data/ai_challenge/tweet_sentiment_extraction/bert-base-uncased'
    OUTPUT_DIR = '/data/ai_challenge/tweet_sentiment_extraction'
    TRAIN_FILE = 'train.csv'
    TEST_FILE = 'test.csv'
    SAVE_MODEL_DIR = 'trained_model'
    MAX_LEN = 128
    TRAIN_BATCH_SIZE = 8
    VALID_BATCH_SIZE = 8
    LOGGING_STEPS = 100
    LEARNING_RATE = 3e-5
    WEIGHT_DECAY = 1e-3
    EPOCHS = 3
    N_SPLITS = 5
    PATIENCE = 2
    SEED = 1234
    DEVICE = torch.device('cuda')
    TOKENIZER = tokenizers.BertWordPieceTokenizer(
        f"{BERT_PATH}/vocab.txt", 
        lowercase=True
    )

In [35]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything(config.SEED)

# Utils

In [36]:
class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class EarlyStopping:
    def __init__(self, patience=7, mode="max", delta=0.001):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):

        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print('Validation score improved ({} --> {}). Saving model!'.format(self.val_score, epoch_score))
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score


def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

# Data Processing

In [37]:
def process_data(tweet, selected_text, sentiment, tokenizer, max_len):
    len_st = len(selected_text)
    idx0 = None
    idx1 = None
    
    # Test again
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[0]):
        if tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break
            
    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
            
    tok_tweet = tokenizer.encode(tweet)
    input_ids_orig = tok_tweet.ids[1: -1]
    tweet_offsets = tok_tweet.offsets[1: -1]
    
    target_idx = [] # offset of token in sentence
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
            
    targets_start = target_idx[0]
    targets_end = target_idx[-1]
    
    sentiment_id = {
        'positive': 3893,
        'negative': 4997,
        'neutral': 8699
    }
    
    input_ids = [101] + [sentiment_id[sentiment]] + [102] + input_ids_orig + [102]
    token_type_ids = [0, 0, 0] + [1] * (len(input_ids_orig) + 1)
    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] * 3 + tweet_offsets + [(0, 0)]
    targets_start += 3
    targets_end += 3
    
    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([0] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
        
    return {
        'ids': input_ids,
        'mask': mask,
        'token_type_ids': token_type_ids,
        'targets_start': targets_start,
        'targets_end': targets_end,
        'orig_tweet': tweet,
        'orig_selected': selected_text,
        'sentiment': sentiment,
        'offsets': tweet_offsets
    }

# Data Loader

In [38]:
class TweetDataset :
    def __init__(self, tweet, sentiment, selected_text):
        self.tweet = tweet
        self.sentiment = sentiment
        self.selected_text = selected_text
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN
    
    def __len__(self):
        return len(self.tweet)
    
    def __getitem__(self, item):
        data = process_data(
            self.tweet[item],
            self.selected_text[item],
            self.sentiment[item],
            self.tokenizer,
            self.max_len
        )
        
        return {
            'ids': torch.tensor(data['ids'], dtype=torch.long),
            'mask': torch.tensor(data['mask'], dtype=torch.long),
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'targets_start': torch.tensor(data["targets_start"], dtype=torch.long),
            'targets_end': torch.tensor(data["targets_end"], dtype=torch.long),
            'orig_tweet': data["orig_tweet"],
            'orig_selected': data["orig_selected"],
            'sentiment': data["sentiment"],
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }

# Model

In [39]:
class TweetModel(transformers.BertPreTrainedModel):
    def __init__(self, conf):
        super(TweetModel, self).__init__(conf)
        self.bert = transformers.BertModel.from_pretrained(config.BERT_PATH, config=conf)
        self.drop_out = nn.Dropout(0.1)
        self.l0 = nn.Linear(768 * 2, 2)
        nn.init.normal_(self.l0.weight, std=0.02)
        
    def forward(self, ids, mask, token_type_ids):
        _, _, out = self.bert(
            ids, 
            attention_mask = mask,
            token_type_ids = token_type_ids
        )
        
        out = torch.cat((out[-1], out[-2]), dim=-1)
        out = self.drop_out(out)
        logits = self.l0(out)

        start_logits, end_logits = logits.split(1, dim=-1)

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

In [40]:
def loss_fn(start_logits, end_logits, start_positions, end_positions):
    loss_fct = nn.CrossEntropyLoss()
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    total_loss = start_loss + end_loss
    
    return total_loss

# Train and Valid Functions

In [41]:
def train_fn(data_loader, model, optimizer, scheduler=None):
    model.train()
    losses = AverageMeter()
    jaccards = AverageMeter()
    
    tk0 = tqdm(data_loader, total=len(data_loader), desc='Train')
    for bi, d in enumerate(tk0):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        sentiment = d["sentiment"]
        orig_selected = d["orig_selected"]
        orig_tweet = d["orig_tweet"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        offsets = d["offsets"]

        ids = ids.to(config.DEVICE, dtype=torch.long)
        token_type_ids = token_type_ids.to(config.DEVICE, dtype=torch.long)
        mask = mask.to(config.DEVICE, dtype=torch.long)
        targets_start = targets_start.to(config.DEVICE, dtype=torch.long)
        targets_end = targets_end.to(config.DEVICE, dtype=torch.long)
        
        model.zero_grad()
        outputs_start, outputs_end = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        
        loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
        outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
        jaccard_scores = []
        
        for px, tweet in enumerate(orig_tweet):
            selected_tweet = orig_selected[px]
            tweet_sentiment = sentiment[px]
            jaccard_score, _ = calculate_jaccard_score(
                original_tweet=tweet,
                target_string=selected_tweet,
                sentiment_val=tweet_sentiment,
                idx_start=np.argmax(outputs_start[px, :]),
                idx_end=np.argmax(outputs_end[px, :]),
                offsets=offsets[px]
            )
            jaccard_scores.append(jaccard_score)
        
        jaccards.update(np.mean(jaccard_scores), ids.size(0))
        losses.update(loss.item(), ids.size(0))
        if bi >= config.LOGGING_STEPS and bi % config.LOGGING_STEPS == 0:
            print(f"Loss: {losses.avg} - Jaccard: {jaccards.avg}")
            losses.reset()
            jaccards.reset()

In [42]:
def calculate_jaccard_score(original_tweet, target_string, sentiment_val, idx_start, idx_end, offsets):
    if idx_end < idx_start:
        idx_end = idx_start
        
    filtered_output  = ""
    for ix in range(idx_start, idx_end + 1):
        filtered_output += original_tweet[offsets[ix][0]: offsets[ix][1]]
        if (ix+1) < len(offsets) and offsets[ix][1] < offsets[ix+1][0]:
            filtered_output += " "
            
    if sentiment_val == 'neutral' or len(original_tweet.split()) < 2:
        filtered_output = original_tweet
        
    jac = jaccard(target_string.strip(), filtered_output.strip())
    
    return jac, filtered_output

def eval_fn(data_loader, model):
    model.eval()
    losses = AverageMeter()
    jaccards = AverageMeter()
    
    with torch.no_grad():
        tk0 = tqdm(data_loader, total=len(data_loader), desc='Valid')
        for bi, d in enumerate(tk0):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            sentiment = d["sentiment"]
            orig_selected = d["orig_selected"]
            orig_tweet = d["orig_tweet"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            offsets = d["offsets"].numpy()

            ids = ids.to(config.DEVICE, dtype=torch.long)
            token_type_ids = token_type_ids.to(config.DEVICE, dtype=torch.long)
            mask = mask.to(config.DEVICE, dtype=torch.long)
            targets_start = targets_start.to(config.DEVICE, dtype=torch.long)
            targets_end = targets_end.to(config.DEVICE, dtype=torch.long)

            outputs_start, outputs_end = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )
            loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
            outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
            outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
            jaccard_scores = []
            for px, tweet in enumerate(orig_tweet):
                selected_tweet = orig_selected[px]
                tweet_sentiment = sentiment[px]
                jaccard_score, _ = calculate_jaccard_score(
                    original_tweet=tweet,
                    target_string=selected_tweet,
                    sentiment_val=tweet_sentiment,
                    idx_start=np.argmax(outputs_start[px, :]),
                    idx_end=np.argmax(outputs_end[px, :]),
                    offsets=offsets[px]
                )
                jaccard_scores.append(jaccard_score)

            jaccards.update(np.mean(jaccard_scores), ids.size(0))
            losses.update(loss.item(), ids.size(0))
            tk0.set_postfix(loss=losses.avg, jaccard=jaccards.avg)
    
    print(f"Validation Jaccard = {jaccards.avg}")
    return jaccards.avg

# Training

In [43]:
def run(fold, train_data_loader, valid_data_loader, train_size, model_config):
    model = TweetModel(conf=model_config)
    model.to(config.DEVICE)
    
    num_train_steps = int(train_size / config.TRAIN_BATCH_SIZE * config.EPOCHS)
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': config.WEIGHT_DECAY},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
    optimizer = AdamW(optimizer_parameters, lr=config.LEARNING_RATE)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=num_train_steps
    )
    
    es = EarlyStopping(patience=config.PATIENCE, mode="max")
    
    for epoch in range(config.EPOCHS):
        train_fn(train_data_loader, model, optimizer, scheduler=scheduler)
        jaccard = eval_fn(valid_data_loader, model)
        es(jaccard, model, model_path=f"{config.OUTPUT_DIR}/{config.SAVE_MODEL_DIR}/model_fold_{fold}.bin")
        if es.early_stop:
            print("Early stopping")
            break

In [44]:
train_df = pd.read_csv(os.path.join(config.DATA_DIR, config.TRAIN_FILE))
train_df = train_df.dropna(how='any', axis=0)
splits = list(StratifiedKFold(n_splits=config.N_SPLITS, shuffle=True, random_state=config.SEED).split(train_df, train_df['sentiment']))

model_config = transformers.BertConfig.from_pretrained(config.BERT_PATH)
model_config.output_hidden_states = True

for fold, (train_idx, val_idx) in enumerate(splits):
    print('---Train Fold {}---'.format(fold))
    
    fold_train_df = train_df.iloc[train_idx, :]
    fold_valid_df = train_df.iloc[val_idx, :]
    
    train_dataset = TweetDataset(
        tweet = fold_train_df.text.values,
        sentiment = fold_train_df.sentiment.values,
        selected_text = fold_train_df.selected_text.values
    )
    
    valid_dataset = TweetDataset(
        tweet = fold_valid_df.text.values,
        sentiment = fold_valid_df.sentiment.values,
        selected_text = fold_valid_df.selected_text.values
    )
    
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size = config.TRAIN_BATCH_SIZE,
        shuffle = True,
        num_workers = 8
    )
    
    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size = config.VALID_BATCH_SIZE,
        shuffle = False,
        num_workers = 8
    )
    
    run(fold=fold, 
        train_data_loader=train_data_loader, 
        valid_data_loader=valid_data_loader, 
        train_size=len(train_idx),
        model_config=model_config
       )

I0425 23:21:03.642301 139864187406080 configuration_utils.py:149] loading configuration file /data/ai_challenge/tweet_sentiment_extraction/bert-base-uncased/config.json
I0425 23:21:03.643000 139864187406080 configuration_utils.py:169] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

I0425 23:21:03.646480 139864187406080 modeling_utils.py:384] loading weights file /data/ai_challenge/tweet_sentiment_extraction/bert-base-uncased/pytorch_model.bin


Train Fold 0


Loss: 4.006890003043826 - Jaccard: 0.5641843551036889
Loss: 2.3982258260250093 - Jaccard: 0.6278097417701818
Loss: 2.1957616502046586 - Jaccard: 0.6605873786543285
Loss: 2.2117417457699777 - Jaccard: 0.6587832252461537
Loss: 2.0294745269417764 - Jaccard: 0.6834536842008329
Loss: 2.000752937793732 - Jaccard: 0.6589861732157594
Loss: 2.0507117620110513 - Jaccard: 0.6529942369698438
Loss: 2.023613204956055 - Jaccard: 0.6436526877577187
Loss: 1.9709315341711044 - Jaccard: 0.6747795378235841
Loss: 2.0475505888462067 - Jaccard: 0.6696222275010871
Loss: 2.0431724429130553 - Jaccard: 0.650858327888883
Loss: 1.9772656041383743 - Jaccard: 0.6585664789287548
Loss: 2.0552616330981253 - Jaccard: 0.6609851099666662
Loss: 1.8740272343158721 - Jaccard: 0.6770454389736977
Loss: 1.765090399980545 - Jaccard: 0.6827987630838354
Loss: 1.9156112080812455 - Jaccard: 0.683015192905799
Loss: 1.91412092640996 - Jaccard: 0.6775231139694191
Loss: 1.8191272270679475 - Jaccard: 0.6838461505192979
Loss: 1.7774360516

Eval Jaccard = 0.6924205063985204
Validation score improved (-inf --> 0.6924205063985204). Saving model!


Loss: 1.6106412092057785 - Jaccard: 0.7209827439982722
Loss: 1.5524682834744454 - Jaccard: 0.7119186405242446
Loss: 1.438403276503086 - Jaccard: 0.7309713882697799
Loss: 1.451978203356266 - Jaccard: 0.7249674359932761
Loss: 1.552498097717762 - Jaccard: 0.7198944857517
Loss: 1.4662634551525116 - Jaccard: 0.730182983043756
Loss: 1.4619652596116066 - Jaccard: 0.7281056459082029
Loss: 1.3672282102704048 - Jaccard: 0.7402388916354566
Loss: 1.486831073462963 - Jaccard: 0.7379813036483414
Loss: 1.4282226225733756 - Jaccard: 0.7444991814764838
Loss: 1.4905841171741485 - Jaccard: 0.7297055939130245
Loss: 1.520411153435707 - Jaccard: 0.7173051083202706
Loss: 1.4854453736543656 - Jaccard: 0.7275581279192997
Loss: 1.5596366119384766 - Jaccard: 0.7230809959096764
Loss: 1.399848034977913 - Jaccard: 0.7368017627452451
Loss: 1.5916595500707627 - Jaccard: 0.7146032801694497
Loss: 1.5352915725111962 - Jaccard: 0.7212640226350238
Loss: 1.4115150326490402 - Jaccard: 0.7497325916256868
Loss: 1.537011525034

...

Eval Jaccard = 0.6972649499515994
Validation score improved (-inf --> 0.6972649499515994). Saving model!


Loss: 1.4557868652414567 - Jaccard: 0.7489423385609864
Loss: 1.511812302172184 - Jaccard: 0.7227037581151647
Loss: 1.2974335381388664 - Jaccard: 0.7570847673353479
Loss: 1.5244905769824981 - Jaccard: 0.7371176018051548
Loss: 1.5194003239274025 - Jaccard: 0.7334389141695135
Loss: 1.49105104804039 - Jaccard: 0.7192172699700697
Loss: 1.5214613628387452 - Jaccard: 0.7137459086039292
Loss: 1.4389581969380378 - Jaccard: 0.729086784588671
Loss: 1.5028460919857025 - Jaccard: 0.733141986306906
Loss: 1.5228749388456344 - Jaccard: 0.7265329354076848
Loss: 1.4801782149076461 - Jaccard: 0.7439439599057631
Loss: 1.4979138001799583 - Jaccard: 0.7310023423958952
Loss: 1.6523909589648247 - Jaccard: 0.7188076184620619
Loss: 1.5042451202869416 - Jaccard: 0.7245283624362451
Loss: 1.482791654765606 - Jaccard: 0.7428477544869171
Loss: 1.4092244058847427 - Jaccard: 0.7511603639728041
Loss: 1.3878144717216492 - Jaccard: 0.7401971154602565
Loss: 1.4409482270479201 - Jaccard: 0.7402760711775456
Loss: 1.52189621

Eval Jaccard = 0.6946055779990511
EarlyStopping counter: 1 out of 2


Loss: 1.0429256942012521 - Jaccard: 0.7934992678614856
Loss: 1.062065132856369 - Jaccard: 0.8007779229648839
Loss: 1.0403044363856315 - Jaccard: 0.7876143928350798
Loss: 0.987800139784813 - Jaccard: 0.7998503529670739
Loss: 1.1669888666272163 - Jaccard: 0.7700310562935353
Loss: 1.0783107428252696 - Jaccard: 0.7854987109534993
Loss: 1.0647250846028329 - Jaccard: 0.800814799658415
Loss: 1.0023440396785737 - Jaccard: 0.7885185737218147
Loss: 1.0596109688282014 - Jaccard: 0.8044258572653699
Loss: 1.0654757928848266 - Jaccard: 0.7946464921392191
Loss: 1.0451233641803264 - Jaccard: 0.8004563033059955
Loss: 1.076701924800873 - Jaccard: 0.784338012910927
Loss: 0.8958338136970997 - Jaccard: 0.8008538094118139
Loss: 1.0498445466160775 - Jaccard: 0.7985606045217132
Loss: 1.106883365213871 - Jaccard: 0.7906696442602396
Loss: 1.0904978999495507 - Jaccard: 0.7819779729714789
Loss: 1.0959253200888635 - Jaccard: 0.781870105732107
Loss: 1.0570912951231002 - Jaccard: 0.7832638525208289
Loss: 1.144183732

Eval Jaccard = 0.6907326772750885
EarlyStopping counter: 2 out of 2
Early stopping


# Testing

In [12]:
test_df = pd.read_csv(os.path.join(config.DATA_DIR, config.TEST_FILE))
test_df.loc[:, 'selected_text'] = test_df.text.values

In [13]:
model_config = transformers.BertConfig.from_pretrained(config.BERT_PATH)
model_config.output_hidden_states = True

I0424 22:58:20.596534 139864187406080 configuration_utils.py:149] loading configuration file /data/ai_challenge/tweet_sentiment_extraction/bert-base-uncased/config.json
I0424 22:58:20.600911 139864187406080 configuration_utils.py:169] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}



In [ ]:
model1 = TweetModel(conf=model_config)
model1.to(config.DEVICE)
model1.load_state_dict(torch.load(f"{config.OUTPUT_DIR}/{config.SAVE_MODEL_DIR}/model_fold_0.bin"))
model1.eval()

model2 = TweetModel(conf=model_config)
model2.to(config.DEVICE)
model2.load_state_dict(torch.load(f"{config.OUTPUT_DIR}/{config.SAVE_MODEL_DIR}/model_fold_1.bin"))
model2.eval()

model3 = TweetModel(conf=model_config)
model3.to(config.DEVICE)
model3.load_state_dict(torch.load(f"{config.OUTPUT_DIR}/{config.SAVE_MODEL_DIR}/model_fold_2.bin"))
model3.eval()

model4 = TweetModel(conf=model_config)
model4.to(config.DEVICE)
model4.load_state_dict(torch.load(f"{config.OUTPUT_DIR}/{config.SAVE_MODEL_DIR}/model_fold_3.bin"))
model4.eval()

model5 = TweetModel(conf=model_config)
model5.to(config.DEVICE)
model5.load_state_dict(torch.load(f"{config.OUTPUT_DIR}/{config.SAVE_MODEL_DIR}/model_fold_4.bin"))
model5.eval()

In [17]:
final_output = []

test_dataset = TweetDataset(
        tweet = test_df.text.values,
        sentiment = test_df.sentiment.values,
        selected_text = test_df.selected_text.values
)

test_data_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size = config.VALID_BATCH_SIZE,
    shuffle = False,
    num_workers = 8
)

with torch.no_grad():
    tk0 = tqdm(test_data_loader, total=len(test_data_loader))
    for bi, d in enumerate(tk0):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        sentiment = d["sentiment"]
        orig_selected = d["orig_selected"]
        orig_tweet = d["orig_tweet"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        offsets = d["offsets"].numpy()

        ids = ids.to(config.DEVICE, dtype=torch.long)
        token_type_ids = token_type_ids.to(config.DEVICE, dtype=torch.long)
        mask = mask.to(config.DEVICE, dtype=torch.long)
        targets_start = targets_start.to(config.DEVICE, dtype=torch.long)
        targets_end = targets_end.to(config.DEVICE, dtype=torch.long)

        outputs_start1, outputs_end1 = model1(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )
        
        outputs_start2, outputs_end2 = model2(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )
        
        outputs_start3, outputs_end3 = model3(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )
        
        outputs_start4, outputs_end4 = model4(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )
        
        outputs_start5, outputs_end5 = model5(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )
        
        outputs_start = (
            outputs_start1 
            + outputs_start2 
            + outputs_start3 
            + outputs_start4 
            + outputs_start5
        ) / 5
        
        outputs_end = (
            outputs_end1 
            + outputs_end2 
            + outputs_end3 
            + outputs_end4 
            + outputs_end5
        ) / 5
        
        outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
        outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()

        for px, tweet in enumerate(orig_tweet):
            selected_tweet = orig_selected[px]
            tweet_sentiment = sentiment[px]
            
            _, output_sentence = calculate_jaccard_score(
                original_tweet=tweet,
                target_string=selected_tweet,
                sentiment_val=tweet_sentiment,
                idx_start=np.argmax(outputs_start[px, :]),
                idx_end=np.argmax(outputs_end[px, :]),
                offsets=offsets[px]
            )
            
            final_output.append(output_sentence)

I0424 23:18:49.669809 139864187406080 modeling_utils.py:384] loading weights file /data/ai_challenge/tweet_sentiment_extraction/bert-base-uncased/pytorch_model.bin


> <ipython-input-17-b83926e6aba2>(26)<module>()
     24     with torch.no_grad():
     25         tk0 = tqdm(test_data_loader, total=len(test_data_loader))
---> 26         for bi, d in enumerate(tk0):
     27             ids = d["ids"]
     28             token_type_ids = d["token_type_ids"]

ipdb> outputs_start.shape
(8, 128)
ipdb> np.argmax(outputs_start)
387
ipdb> np.argmax(outputs_start[0, :])
3
ipdb> q


BdbQuit: 

In [23]:
sample = pd.read_csv(os.path.join(config.DATA_DIR, "sample_submission.csv"))
sample.loc[:, 'selected_text'] = final_output
sample.to_csv(os.path.join(config.OUTPUT_DIR, "submission.csv"), index=False)

sample.head()

,textID,selected_text
0,f87dea47db,http://twitpic.com/67ezh last session day the of
1,96d74cb729,good
2,eee518ae67,shame! a such
3,01082688c6,happy bday!
4,33987a8ee5,it!! like i
